## The Logistics Commander (CoT & ToT)

In [2]:
import pandas as pd
import sys
sys.path.append('..')

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model
from IPython.display import Markdown, display

In [4]:
file_path = "../data/incidents.txt"

# Read file as raw text
with open(file_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# First line is header
header = lines[0]
rows = lines[1:]

rows

['1  | 08:00 AM| Gampaha | 4      | 20-40  | Water     | "Thirsty but safe on roof. Water level stable."',
 '2  | 08:15 AM| Ja-Ela  | 1      | 75     | Insulin   | "Diabetic, missed dose yesterday. Feeling faint."',
 '3  | 08:20 AM| Ragama  | 2      | 10, 35 | Rescue    | "Water approaching neck level. Child is crying."']

In [6]:
incidents = []

for row in rows:
    parts = [p.strip() for p in row.split("|")]

    incident = {
        "id": int(parts[0]),
        "time": parts[1],
        "area": parts[2],
        "people": int(parts[3]),
        "ages": parts[4],
        "main_need": parts[5],
        "message": parts[6].strip('"')
    }

    incidents.append(incident)

incidents


[{'id': 1,
  'time': '08:00 AM',
  'area': 'Gampaha',
  'people': 4,
  'ages': '20-40',
  'main_need': 'Water',
  'message': 'Thirsty but safe on roof. Water level stable.'},
 {'id': 2,
  'time': '08:15 AM',
  'area': 'Ja-Ela',
  'people': 1,
  'ages': '75',
  'main_need': 'Insulin',
  'message': 'Diabetic, missed dose yesterday. Feeling faint.'},
 {'id': 3,
  'time': '08:20 AM',
  'area': 'Ragama',
  'people': 2,
  'ages': '10, 35',
  'main_need': 'Rescue',
  'message': 'Water approaching neck level. Child is crying.'}]

In [7]:
df_incidents = pd.DataFrame(incidents)
df_incidents

,id,time,area,people,ages,main_need,message
0,1,08:00 AM,Gampaha,4,20-40,Water,Thirsty but safe on roof. Water level stable.
1,2,08:15 AM,Ja-Ela,1,75,Insulin,"Diabetic, missed dose yesterday. Feeling faint."
2,3,08:20 AM,Ragama,2,"10, 35",Rescue,Water approaching neck level. Child is crying.


In [8]:
def parse_ages(age_str):
    nums = [int(x) for x in age_str.replace("-", ",").split(",")]
    return min(nums), max(nums)

df_incidents[["min_age", "max_age"]] = df_incidents["ages"].apply(
    lambda x: pd.Series(parse_ages(x))
)

df_incidents

,id,time,area,people,ages,main_need,message,min_age,max_age
0,1,08:00 AM,Gampaha,4,20-40,Water,Thirsty but safe on roof. Water level stable.,20,40
1,2,08:15 AM,Ja-Ela,1,75,Insulin,"Diabetic, missed dose yesterday. Feeling faint.",75,75
2,3,08:20 AM,Ragama,2,"10, 35",Rescue,Water approaching neck level. Child is crying.,10,35


In [12]:
# CoT auto-routes to reasoning model
reasoning_model = pick_model('groq', 'cot')
print(f'Using reasoning model: {reasoning_model}')

client_reasoning = LLMClient('groq', reasoning_model)


problem = incidents

# Additional guidance 
instruction = """You are a disaster logistics analyst.

You must evaluate the following incident and assign a
priority score using step-by-step reasoning internally.

-----------------------------------
SCORING RULES:
- Base Score: 5
- +2 if any person is younger than 5 or older than 60
- +3 if Main Need is "Rescue" (immediate life threat)
- +1 if Main Need is "Insulin" or urgent medicine
- Maximum possible score: 10

-----------------------------------
INCIDENT DETAILS:
Area: {area}
People: {people}
Ages: {ages}
Main Need: {main_need}
Message: "{message}"

-----------------------------------
IMPORTANT:
- You MUST reason step-by-step internally
- You MUST NOT show your reasoning
- You MUST output ONLY the final score

-----------------------------------
OUTPUT FORMAT (STRICT):
{id} Score: X/10
"""

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='disaster logistics analyst',
    problem=problem
)

# Combine problem with instruction 
full_prompt = f"""text: {prompt_text}

instruction: {instruction}"""

messages = [{'role': 'user', 'content': full_prompt}]
response = client_reasoning.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

print('CoT Response :')
print('=' * 80)
print(response['text'])
print('=' * 80)
log_llm_call('groq', reasoning_model, 'cot', response['latency_ms'], response['usage'])

Using reasoning model: openai/gpt-oss-120b
CoT Response :
1 Score: 5/10
2 Score: 8/10
3 Score: 8/10
